Boundary Layer Thickness on a Flat Plate
===

This is a tool for a quick estimation of the boundary layer thickness on a flat plate.

The momentum loss thickness of a laminar boundary layer $\delta_l$ (Blasius solution):
$$ \delta_l = 0.664 \cdot \sqrt{\frac{\nu \cdot x}{u}} $$

For a turbulent flow:
$$ \delta_t = 0.0142 \cdot x \cdot \left(\frac{u\cdot x}{\nu}\right)^\left(-\frac{1}{7}\right) $$

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import ipywidgets as widgets

import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import root_scalar

import sys
sys.path.append('../_lib')
import gas

nu = 151e-7
x = np.linspace(1e-20,31.4e-3,100)

c1 = 1


                  
@np.vectorize
def momentum_boundarylayer(x, u, gas):
    rho = gas.density
    nu = gas.kinematic_viscosity
    
    fdelta_laminar = lambda x: 0.664*np.sqrt(nu*x/u)
    fdelta_turbulent = lambda x: 0.0142*(u*x/nu)**(-1/7)*x
    
    try:
        boundarylayer_thickness = [np.max(fdelta_laminar(xi), fdelta_turbulent(xi)) for xi in x]
    except TypeError:
        boundarylayer_thickness = np.max([fdelta_laminar(x), fdelta_turbulent(x)])
    
    #find where laminar and turbulent boundary layer thickness intersect
    sol = root_scalar(lambda x: fdelta_laminar(x)-fdelta_turbulent(x), bracket=[1e-6,10e6])
    transition_position = sol.root
    
    return(boundarylayer_thickness,
           transition_position)
    
nu = 151e-7
Lchar = 0.032

Re = lambda u: Lchar*u/nu
    
g = gas.state('Air', 298.15, 101325)

u = np.linspace(0.1,50, 1000)
bl = momentum_boundarylayer(0.032, u, g)[0]
#bl = np.array([np.array(momentum_boundarylayer(0.032, vel, g)) for vel in u])


#plt.plot(Re(u),bl*1000)
#plt.grid()
#plt.xlabel('Reynolds number')
#plt.ylabel('boundary layer thickness in mm')
    
    

In [40]:
def fplot(platelength, u, pressure, temperature, gasname):
    g = gas.state(gasname, 273.15+temperature, pressure*1e5)
    
    
    plt.figure()
    #print(xmax, u, gas.gasname)
    x = np.linspace(0.0000001, platelength, num=1000)
    
    bl, transition = momentum_boundarylayer(x, u, g)
    transition_bl, _ =  momentum_boundarylayer(transition[0], u, g)
    #print(x, bl)
    plt.plot(x, bl*1000, label='boundary layer thickness')
    plt.plot(transition[0], transition_bl*1000, 'o', label='laminar-turbulent transition')
    plt.xlabel('length of flat plate in m')
    plt.ylabel('boundary layer thickness in mm')
    plt.grid()
    plt.xlim([0, platelength])
    plt.ylim([0, np.max(bl)*1000])
    plt.title('')
    plt.legend()
    plt.show()

In [41]:
unit_MPa = widgets.Label(value='\(\mathrm{MPa}\)')
unit_C = widgets.Label(value='\(\mathrm{°C}\)')
unit_m = widgets.Label(value='\(\mathrm{m}\)')
unit_mpers = widgets.Label(value='\(\mathrm{m/s}\)')

pressure_slider = widgets.FloatSlider(value=0.101325, min=0.1, max=15, step=0.1,
                                      description='pressure:', readout_format='.1f')


temperature_slider = widgets.IntSlider(value=20, min=-20, max=50, step=1,
                                       description='temperature:')



#gas_dropdown = widgets.Dropdown(options={'Air, 1atm, 20C': gas.state('Air', 298.15, 101325),
#                                    'Air, 17bar, 20C': gas.state('Air', 298.15, 17e5),
#                                    'Air, 50bar, 20C': gas.state('Air', 298.15, 5e6),
#                                    'N2': gas.state('N2', 298.15, 101325),
#                                    'H2': gas.state('H2', 298.15, 101325)})
gas_dropdown = widgets.Dropdown(options=['Air', 'N2', 'H2', 'CH4'])

platelength_slider = widgets.FloatSlider(description='plate length:',
                                         readout_format='.3f',
                                         min=0.001, max=1, step=0.001, value=0.032)
velocity_slider = widgets.FloatSlider(description='velocity:',
                                         min=0.25, max=100, step=0.25, value=25.0)


thermostate = widgets.VBox([
    widgets.HBox([pressure_slider, unit_MPa]),
    widgets.HBox([temperature_slider, unit_C]),
    widgets.HBox([widgets.Label('gas:'), gas_dropdown])
])

flow = widgets.VBox([
    widgets.HBox([platelength_slider, unit_m]),
    widgets.HBox([velocity_slider, unit_mpers]),
                 
])

accordion = widgets.Accordion(children=[thermostate, flow])
accordion.set_title(0, 'Thermodynamic State')
accordion.set_title(1, 'Flat Plate')


interactive_plot = interactive_output(fplot,
                               {'platelength': platelength_slider,
                                'u': velocity_slider,
                                'pressure': pressure_slider,
                                'temperature': temperature_slider,
                                'gasname': gas_dropdown})

display(accordion, interactive_plot)

Accordion(children=(VBox(children=(HBox(children=(FloatSlider(value=0.101325, description='pressure:', max=15.…

Output()